### Generating a toy-case topology

In [436]:
import numpy as np 

In [437]:
# Let's generate a toy topology for our example

nodes = [i for i in range(7)]
edges = [
    (0,1),
    (0,3),
    (0,6),
    (1,2),
    (1,5),
    (2,4),
    (4,6),
    (5,6)
]

V = 7
E = len(edges)

d = 3                                           # Node and edges stalks dimension

F = {
    e:{
        e[0]:np.random.randn(d,d),
        e[1]:np.random.randn(d,d)
        } 
        for e in edges
    }                                           # Incidency linear maps

# Sheaf representation 

# Coboundary map

B = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = F[edge][u]
    B_v = F[edge][v]

    B[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

# Sheaf Laplacian

L_f = B.T @ B

# Generating a smooth signals dataset 

*(from Hansen J., "Learning sheaf Laplacians from smooth signals")* 

In order to retrieve a dataset of smoothsignals, first of all we sample random gaussians vectors on the nodes of the graph. Then we smooth them according to their expansion in terms of the eigenvectors of the sheaf Laplacian $L_0$.

So let's firstly define a dataset of random gaussian vectors. 

In [438]:
N = 100
X = np.random.randn(V*d,N)

Now we'll use the Fourier-domain embedded in the Laplacian spectrum. 

We'll consider a Tikhonov inspired procedure where we firstly project our dataset over the space spanned by the eigenvectors of the sheaf laplacian: namely $U$ the matrix collecting this eigenvectors we have 
\begin{equation}
    \hat{x} = U^T x
\end{equation}

So that defining $h(\lambda) = \frac{1}{1 + 10\lambda}$ and $H = \mathrm{diag}\{h(\lambda)\}_{\lambda}$, we now have

\begin{equation}
    \hat{y} = H(\Lambda) \hat{x}
\end{equation}

and finally our dataset is just reprojected back into the vertex domain:

\begin{equation}
    y = U H(\Lambda) \hat{x} = U H(\Lambda) U^T x
\end{equation}

In [439]:
Lambda, U = np.linalg.eig(L_f)
H = 1/(1 + 10*Lambda)

In [440]:
Y = U @ np.diag(H) @ U.T @ X

Y += np.random.normal(0, 10e-2, size=Y.shape)

In [441]:
np.trace(X.T @ L_f @ X)

15481.183983912217

In [442]:
np.trace(Y.T @ L_f @ Y)

175.8384078504842

____________

In [443]:
from itertools import combinations
from tqdm import tqdm

In [444]:
def premultiplier(Xu, Xv):
    uu = np.linalg.inv(Xu @ Xu.T)
    uv = Xu @ Xv.T
    vv = np.linalg.inv(Xv @ Xv.T)
    vu = Xv @ Xu.T

    return (uu, uv, vv, vu)

def chi_u(uu, uv, vv, vu):

    return ((uu @ uv - np.eye(uu.shape[0])) @ vv @ np.linalg.inv(vu @ uu @ uv @ vv - np.eye(uu.shape[0])) @ vu - np.eye(uu.shape[0])) @ uu

def chi_v(uu, uv, vv, vu):

    return (uu @ uv - np.eye(uu.shape[0])) @ vv @ np.linalg.inv(vu @ uu @ uv @ vv - np.eye(uu.shape[0]))

In [445]:
T = 0

maps = {
    edge : {
        edge[0] : np.zeros((d_e,d)),
        edge[1] : np.zeros((d_e,d))
    }
for edge in combinations(nodes, 2)
}

In [446]:
for e in tqdm(combinations(nodes,2)):
    u = e[0]
    v = e[1]

    X_u = Y[u*d:(u+1)*d,:]
    X_v = Y[v*d:(v+1)*d,:]
    uu, uv, vv, vu = premultiplier(X_u, X_v)

    maps[e][u] = chi_u(uu, uv, vv, vu)
    maps[e][v] = chi_u(uu, uv, vv, vu)
    
    T += np.trace(maps[e][u]) + np.trace(maps[e][v])

21it [00:00, 2624.49it/s]


In [447]:
import sys
sys.path.append(r'C:\Users\Leonardo\Documents\GitHub\GraphCellSheaf\LaplacianLearning\SharingOptimization')

In [448]:
from data_utils import *

In [449]:
for N in range(1,100):
    maps_ = {
        edge : {
            edge[0] : N/T * maps[edge][edge[0]],
            edge[1] : N/T * maps[edge][edge[1]]
        }
        for edge in combinations(nodes, 2)
    }
    print(N,reconstructed_laplacian_metrics(len(nodes), edges, d, maps_, Y, L_f))

1 {'AvgEntryWiseED_L2': 0.11488197608032148, 'AvgEntryWiseED_L1': 0.05006601554304098, 'SparsityAccuracy': 0.5}
2 {'AvgEntryWiseED_L2': 0.11487724807514738, 'AvgEntryWiseED_L1': 0.050065773841368004, 'SparsityAccuracy': 0.5}
3 {'AvgEntryWiseED_L2': 0.11486936909521005, 'AvgEntryWiseED_L1': 0.05006537100265332, 'SparsityAccuracy': 0.5}
4 {'AvgEntryWiseED_L2': 0.1148583406839833, 'AvgEntryWiseED_L1': 0.05006480702300726, 'SparsityAccuracy': 0.5}
5 {'AvgEntryWiseED_L2': 0.11484416500321977, 'AvgEntryWiseED_L1': 0.05006408189698398, 'SparsityAccuracy': 0.5}
6 {'AvgEntryWiseED_L2': 0.11482684483384031, 'AvgEntryWiseED_L1': 0.050063195617581234, 'SparsityAccuracy': 0.5}
7 {'AvgEntryWiseED_L2': 0.11480638357707836, 'AvgEntryWiseED_L1': 0.05006214817624006, 'SparsityAccuracy': 0.5}
8 {'AvgEntryWiseED_L2': 0.11478278525587911, 'AvgEntryWiseED_L1': 0.05006093956284431, 'SparsityAccuracy': 0.5}
9 {'AvgEntryWiseED_L2': 0.114756054516554, 'AvgEntryWiseED_L1': 0.05005956976572024, 'SparsityAccuracy'

_______________